### setup

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import pageviewapi
import datetime as dt
from bs4 import BeautifulSoup
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="PU52xuCQowgE5BFprvdnpFXzd5Ql9ISOSLBk58eNR2ykPztw0yC5fpqLTfsZdhyR"
)

In [5]:
# scope = ['https://spreadsheets.google.com/feeds']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-integration-349314-25735d35924b.json', scope)
# gc = gspread.authorize(credentials)

In [6]:
df = pd.read_json(
    "http://tropical.atmos.colostate.edu/Realtime/archives/northatlantic.json",
    orient="index",
).reset_index()

In [7]:
df.columns = [
    "year",
    "named_storms",
    "named_storm_days",
    "hurricanes",
    "hurricane_days",
    "major_hurricanes",
    "major_hurricane_days",
    "accumulated_cyclone_energy",
]

In [8]:
days = df.melt(
    id_vars=["year"],
    value_vars=["named_storm_days", "hurricane_days", "major_hurricane_days"],
)

In [9]:
storms = df.melt(
    id_vars=["year"],
    value_vars=["named_storms", "hurricanes", "major_hurricanes"],
)

In [10]:
alt.Chart(days).mark_area().encode(x="year:O", y="value", color=alt.Color("variable:N"))

alt.Chart(...)

In [11]:
alt.Chart(storms).mark_area(opacity=0.6).encode(
    x="year:O", y="value", color=alt.Color("variable:N")
)

alt.Chart(...)

In [12]:
df.sort_values("year", ascending=True)
df["hurricane_days_rolling_average"] = df["hurricane_days"].rolling(10).mean()
df["hurricanes_rolling_average"] = df["hurricanes"].rolling(10).mean()

In [13]:
df.sort_values("year", ascending=True)
df["namedstorm_days_rolling_average"] = df["named_storm_days"].rolling(10).mean()
df["namedstorm_rolling_average"] = df["named_storms"].rolling(10).mean()

In [14]:
hurricanes = df.melt(
    id_vars="year",
    value_vars=["hurricanes_rolling_average", "hurricane_days_rolling_average"],
)

In [19]:
named = df.melt(
    id_vars="year",
    value_vars=["namedstorm_days_rolling_average", "namedstorm_rolling_average"],
)

In [20]:
alt.Chart(named).mark_line().encode(
    x="year:O", y="value", color=alt.Color("variable:N")
)

alt.Chart(...)

In [30]:
named_wide = named.pivot(index="year", columns="variable", values="value")

In [37]:
line = (
    alt.Chart(named_wide)
    .mark_line()
    .encode(x=alt.X("year:O"), y=alt.Y("namedstorm_rolling_average"))
)

area = (
    alt.Chart(named_wide)
    .mark_area()
    .encode(x=alt.X("year:O"), y=alt.Y("namedstorm_days_rolling_average"))
)

line + area

alt.LayerChart(...)

In [33]:
### can i make area and line chart here
alt.Chart(named_wide).mark_line().encode(
    x=alt.X("year:O"), y=alt.Y("namedstorm_rolling_average")
).mark_area().encode(x=alt.X("year:O"), y=alt.Y("namedstorm_days_rolling_average"))

alt.Chart(...)

In [34]:
base = alt.Chart(named_wide).encode(alt.X("year:O", axis=alt.Axis(title=None)))


area = base.mark_area(opacity=0.3, color="#57A44C").encode(
    alt.Y("namedstorm_rolling_average")
)

line = base.mark_line(stroke="#5276A7", interpolate="monotone").encode(
    alt.Y("namedstorm_days_rolling_average")
)

alt.layer(area, line)

alt.LayerChart(...)

In [43]:
# dw.create_chart("named storms", chart_type="d3-lines", data=named_wide)
chart_id = "oH4CD"

In [42]:
named_wide = named_wide.reset_index()

In [44]:
dw.add_data(chart_id=chart_id, data=named_wide)

<Response [204]>